In [1]:
#!tar -xzf multilabel_classification_edu7.tgz
#!pip install -U peft
#!unzip 2024-03-08_data_multiple-fields_german-only-20240319T075025Z-001.zip
#!pip install threshold-optimizer
#!pip install scikit-learn
#!pip install -U transformers
#!pip install parfor

In [2]:
import torch
print (torch.cuda.device_count())
print (torch.backends.mps.is_available())
from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig, AutoTokenizer
import csv
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

import numpy as np
import torch.nn.functional as F
from sklearn.metrics import classification_report
import pickle
import gc
from peft import PeftModel, PeftConfig


2
False


/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [3]:
from huggingface_hub import login
from huggingface_hub import hf_hub_download
hf_token = "put your token here" 
login(hf_token) 

In [6]:
#OUTPUTDIR='./multilabel_classification_edu7/'
#peft_model_id = OUTPUTDIR
device_map = "auto"

In [22]:
peft_model_id = "joergwa/Edux2"
#peft_model_id = OUTPUTDIR
config = PeftConfig.from_pretrained(peft_model_id)
model2 = AutoModelForSequenceClassification.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device_map,
    num_labels=586,
#    force_download=True
#    device="cuda:0"
)
model2 = PeftModel.from_pretrained(model2, peft_model_id, force_download=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
score_params = torch.load(OUTPUTDIR + 'score-params_edu.pt', map_location="cuda:0") 
model2.score.load_state_dict(score_params)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/64.2M [00:00<?, ?B/s]

<All keys matched successfully>

In [23]:
datadir = "2024-03-08_data_multiple-fields_german-only"
testfile = datadir + "/test_data.csv"
with open(testfile, newline='') as csvfile:
    fulldata = list(csv.reader(csvfile, delimiter=','))        
    header_row = fulldata.pop(0)

    
testdata=[]
for line in fulldata:   
    datum=[]
    datum.append(line[1])
    for elm in line[2:]:
        datum += list(elm)
    testdata.append(datum)
print (len(testdata))    

headerfile = datadir + "/test_metadata.csv"  # should be the same like train_metadata.csv

metadataList=[]
with open(headerfile, newline='') as csvfile:
    metadata = list(csv.reader(csvfile, delimiter=','))        
    metadata.pop(0)
    metadataList.append(metadata)
    
    
#print (metadataList)
labels={}
header = [header_row[2]] 
for m in metadata:
    #header.append( m[2] + " (" + m[3]+ ")")  # use 2 for uris
    header.append( m[2] )  # use 2 for uris
    print (m[3])
    labels[m[2]]=m[3]

header += header_row[8:]

for h in header:
    if not h in labels:
        labels[h]=h
print ( len(header)) 


def id2label(id):
    return header[int(id)]



17735
Elementarbereich
Hochschule
Berufliche Bildung
Erwachsenenbildung
Fortbildung
Förderschule
Sekundarstufe II
Primarstufe
Fernunterricht
Sekundarstufe I
Autor/in
Eltern
andere
Berater/in
Lerner/in
Lehrer/in
Verwaltung
Türkisch
Pädagogik
Metalltechnik
Sport
Zeitgemäße Bildung
Biologie
Latein
Weiterbildung
Berufliche Bildung
Sozialpädagogik
Physik
Gesellschaftskunde
Arbeitssicherheit
Italienisch
Politik
Werken
Allgemein
Russisch
Englisch
Informatik
Darstellendes Spiel
Gesundheit
Deutsch
Arbeitslehre
Geschichte
Sonderpädagogik
Medienbildung
Umweltgefährdung, Umweltschutz
Wirtschaft und Verwaltung
Open Educational Resources
Spanisch
Elektrotechnik
Musik
Nachhaltigkeit
Astronomie
Französisch
Ernährung und Hauswirtschaft
Psychologie
Hauswirtschaft
Geografie
Kunst
Religion
Deutsch als Zweitsprache
Mathematik
Interkulturelle Bildung
Philosophie
Wirtschaftskunde
Sachunterricht
Chemie
Ethik
Mediendidaktik
MINT
Sonstiges
Runden und Überschlag
Aufstellen von Reaktionsgleichungen
Ökosysteme, Na

In [24]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

In [17]:
def numTensors586():
    tensors=0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                if obj.size()[1]==586:
                    #print(type(obj), obj.size())
                    tensors+=1
        except:
            pass
    return tensors

In [18]:
print (numTensors586())
        
gc.collect()
torch.cuda.empty_cache()

2


In [19]:
datadir = "2024-03-08_data_multiple-fields_german-only"
testfile = datadir + "/test_data.csv"
with open(testfile, newline='') as csvfile:
    fulldata = list(csv.reader(csvfile, delimiter=','))        
    header_row = fulldata.pop(0)

    
testdata=[]
for line in fulldata:   
    datum=[]
    datum.append(line[1])
    for elm in line[2:]:
        datum += list(elm)
    testdata.append(datum)
print (len(testdata))    


17735


In [20]:
headerfile = datadir + "/test_metadata.csv"  # should be the same like train_metadata.csv

metadataList=[]
with open(headerfile, newline='') as csvfile:
    metadata = list(csv.reader(csvfile, delimiter=','))        
    metadata.pop(0)
    metadataList.append(metadata)
    
    
#print (metadataList)
labels={}
header = [header_row[2]] 
for m in metadata:
    #header.append( m[2] + " (" + m[3]+ ")")  # use 2 for uris
    header.append( m[2] )  # use 2 for uris
    print (m[3])
    labels[m[2]]=m[3]

header += header_row[8:]

for h in header:
    if not h in labels:
        labels[h]=h
print ( len(header)) 


def id2label(id):
    return header[int(id)]

Elementarbereich
Hochschule
Berufliche Bildung
Erwachsenenbildung
Fortbildung
Förderschule
Sekundarstufe II
Primarstufe
Fernunterricht
Sekundarstufe I
Autor/in
Eltern
andere
Berater/in
Lerner/in
Lehrer/in
Verwaltung
Türkisch
Pädagogik
Metalltechnik
Sport
Zeitgemäße Bildung
Biologie
Latein
Weiterbildung
Berufliche Bildung
Sozialpädagogik
Physik
Gesellschaftskunde
Arbeitssicherheit
Italienisch
Politik
Werken
Allgemein
Russisch
Englisch
Informatik
Darstellendes Spiel
Gesundheit
Deutsch
Arbeitslehre
Geschichte
Sonderpädagogik
Medienbildung
Umweltgefährdung, Umweltschutz
Wirtschaft und Verwaltung
Open Educational Resources
Spanisch
Elektrotechnik
Musik
Nachhaltigkeit
Astronomie
Französisch
Ernährung und Hauswirtschaft
Psychologie
Hauswirtschaft
Geografie
Kunst
Religion
Deutsch als Zweitsprache
Mathematik
Interkulturelle Bildung
Philosophie
Wirtschaftskunde
Sachunterricht
Chemie
Ethik
Mediendidaktik
MINT
Sonstiges
Runden und Überschlag
Aufstellen von Reaktionsgleichungen
Ökosysteme, Natur- u

In [21]:
print (labels)

{'http://w3id.org/openeduhub/vocabs/educationalContext/elementarbereich': 'Elementarbereich', 'http://w3id.org/openeduhub/vocabs/educationalContext/hochschule': 'Hochschule', 'http://w3id.org/openeduhub/vocabs/educationalContext/berufliche_bildung': 'Berufliche Bildung', 'http://w3id.org/openeduhub/vocabs/educationalContext/erwachsenenbildung': 'Erwachsenenbildung', 'http://w3id.org/openeduhub/vocabs/educationalContext/fortbildung': 'Fortbildung', 'http://w3id.org/openeduhub/vocabs/educationalContext/foerderschule': 'Förderschule', 'http://w3id.org/openeduhub/vocabs/educationalContext/sekundarstufe_2': 'Sekundarstufe II', 'http://w3id.org/openeduhub/vocabs/educationalContext/grundschule': 'Primarstufe', 'http://w3id.org/openeduhub/vocabs/educationalContext/fernunterricht': 'Fernunterricht', 'http://w3id.org/openeduhub/vocabs/educationalContext/sekundarstufe_1': 'Sekundarstufe I', 'http://w3id.org/openeduhub/vocabs/intendedEndUserRole/author': 'Autor/in', 'http://w3id.org/openeduhub/voc

In [ ]:
y_true = []
y_probs = []
y_pred = []

In [ ]:
target_names = header
threshold=0.1

cnt=0;
with torch.no_grad():
  for d in testdata:
    cnt+=1
    if cnt<len(y_pred):
        continue
    #gc.collect()
    #torch.cuda.empty_cache()
    text=d[0]
    text=text[0:4000]
    #print (text)
    classes = d[1:]
    res = [eval(i) for i in classes]
    y_true.append(res)
    #print (res)
    print ("Text", len(text))
    inputs = tokenizer(text, return_tensors="pt")
    output = model2(**inputs).logits 
    probabilities_scores = F.softmax(output.float().detach(), dim = -1).numpy()[0]
    #probabilities_scores = [0.3, 0.1, 0.2, 0.0, 0.0, 0.0123, 0.3, 0.1, 0.0, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0]    
    output.detach().cpu()
    #model2(**inputs).logits.detach().cpu()
    del output
    #del model2(**inputs).logits
    torch.cuda.synchronize()    
    
    print ("Tensors", numTensors586())
    y_probs.append(probabilities_scores)    
    if cnt%100==0:
        print ("CNT:", cnt)
        gc.collect()
        torch.cuda.empty_cache()
        y_pred = []
        
        for i in y_probs:
            arr = np.array(i)
            new_arr = list(np.where(arr > threshold, 1, 0))
            #print(new_arr)  # 👉️ [0 0 0 1 0 1]
            y_pred.append(new_arr)
        #print(classification_report(y_true, y_pred, target_names=target_names))
        pickle.dump(y_true, open("y_true.pik", 'wb'))
        pickle.dump(y_probs, open("y_probs.pik", 'wb'))
        pickle.dump(y_pred, open("y_pred.pik", 'wb'))
    

In [ ]:
y_pred = []
threshold=0.05
for i in y_probs:
    arr = np.array(i)
    new_arr = list(np.where(arr > threshold, 1, 0))
    #print(new_arr)  # 👉️ [0 0 0 1 0 1]
    y_pred.append(new_arr)

print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
print (len(y_probs[0]))

In [ ]:
import pandas as pd

In [ ]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
        
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    #print (fpr, tpr, threshold)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 


In [ ]:
from parfor import parfor

@parfor(range(len(y_probs[0])))
def optClass(classnum):
    maxif = 0.0
    maxit = 0.0
    for t in range(1000):
        threshold = 1.0/(t+0.000001)    
        TP = 0   
        TN = 0   
        FN = 0   
        FP = 0   
        for i in range(len(y_probs)):
            true = y_true[i][classnum]
            pred = y_probs[i][classnum]
            if pred>threshold:
                pred=1
            else:
                pred=0

            if true==1 and pred==1:
                TP+=1
            if true==0 and pred==0:
                TN+=1
            if true==0 and pred==1:    
                FP+=1
            if true==1 and pred==0:    
                FN+=1

        nenner = (2*TP + FN + FP)
        if nenner == 0:
            f = 0
        else:
            f = 2.0 * TP / nenner
        #print (TP, TN, FN, FP, f, threshold, maxif)  
        
        if f > maxif:
            maxif=f
            maxit=threshold
    #thresholds.append(maxit)
    return [classnum, maxit]

#thresholds=[]
#for classnum in range(len(y_probs[0])):     
    
result = optClass   
print (result)    
#    break

In [ ]:
y_pred=[]

for i in y_probs:
    arr = i    
    #print (arr[0])
    new_arr=[]
    for j in range(len(arr)):  
        thres = result[j]
        if arr[j] > thres[1]:
            new_arr.append(1)
        else:
            new_arr.append(0)
  
    y_pred.append(new_arr)



In [ ]:
print(classification_report(y_true, y_pred, target_names=header, zero_division=np.nan))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

def get_accuracies(true_labels, predictions):
    #https://scikit-learn.org/stable/modules/generated/sklearn.metrics.multilabel_confusion_matrix.html
    cm = multilabel_confusion_matrix(true_labels, predictions)
    total_count = true_labels.shape[0]
    accuracies = []
    for i in range(true_labels.shape[1]):
        true_positive_count = np.sum(cm[i,1,1]).item()
        true_negative_count = np.sum(cm[i,0,0]).item()
        accuracy = (true_positive_count + true_negative_count) / total_count
        accuracies.append(accuracy)
    return accuracies

accs = get_accuracies(np.array(y_true), np.array(y_pred))

In [ ]:
report = classification_report(y_true, y_pred, target_names=header, zero_division=np.nan, output_dict=True)

In [ ]:
idx=0
for i in report:
    if idx<len(accs):
        report[i]['accuracy']=accs[idx]
    idx+=1
    
for i in report:
    if 'accuracy' in report[i]:
        print (labels[i], "\t", report[i]['accuracy'], "\t", report[i]['precision'], "\t", report[i]['recall'], "\t", report[i]['f1-score'],  "\t", report[i]['support'] , "\t", i)

In [28]:
text_input = "Dieses youtube Video zeigt eine Einführung in das geometrische Analysieren von Dreiecken Es wird erklärt wie sich die Seiten zueinander verhalten. Der Satz des Pythagoras wird angewandt. Der Film ist für Schüler der 5. bis 7. Klasse bestimmt."
#text_input = "Mathematik zahlen rechnen und alles was damit zu tun hat"
inputs = tokenizer(text_input , return_tensors="pt", padding=True)
print (result[0][1], result[1][1] )

with torch.no_grad():
    model2.config.pad_token_id = model2.config.eos_token_id
    output = model2(**inputs).logits
    print (output[0][0:10])
    probabilities_scores = F.softmax(output.cpu().float().detach(), dim = -1).numpy()[0]
    
    for i in range(len(probabilities_scores)):
        if (probabilities_scores[i] > result[i][1]/10 and result[i][1]!=0.0): 
            print(probabilities_scores[i], result[i][1],  id2label(i))

0.005154639148687427 0.0036101082902162155
tensor([ -1.0537, -11.3438,  -6.2266,  -6.8867,  -8.9297,  -9.8047,  -7.8711,
         -1.4102,  -5.0820,  -9.5469], dtype=torch.float16)
0.00020187582 0.00100200400701202 http://w3id.org/openeduhub/vocabs/educationalContext/sekundarstufe_2
0.0015173833 0.0010030090260752167 http://w3id.org/openeduhub/vocabs/educationalContext/sekundarstufe_1
0.0047964025 0.001007049344403777 http://w3id.org/openeduhub/vocabs/intendedEndUserRole/learner
0.0032581107 0.0013245033095039692 http://w3id.org/openeduhub/vocabs/intendedEndUserRole/teacher
0.98830485 0.0016611295653469608 http://w3id.org/openeduhub/vocabs/discipline/380
0.0006409338 0.005076142106212477 http://w3id.org/openeduhub/vocabs/new_lrt/7a6e9608-2554-4981-95dc-47ab9ba924de
0.0006782017 0.0039525691543376715 http://w3id.org/openeduhub/vocabs/new_lrt/a0218a48-a008-4975-a62a-27b1a83d454f
